In [6]:
from typing import Dict, TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain.llms import OpenAI

from langchain.output_parsers import CommaSeparatedListOutputParser
import random
import time

llm = OpenAI()
debate_topic= "Should Data Scientists write backend and API codes as well?"

output_parser = CommaSeparatedListOutputParser()
output = llm("I wish to have a debate on {}. What would be the fighting sides called? Output just the names and nothing else as comma separated list".format(debate_topic))
classes = output_parser.parse(output)

print(classes)


class GraphState(TypedDict):
    classification: Optional[str] = None
    history: Optional[str] = None
    current_response: Optional[str] = None
    count: Optional[int]=None
    results: Optional[str]=None
    greeting: Optional[str]=None

workflow = StateGraph(GraphState)

prefix_start= '''You are in support of {} . You are in a debate with {} over the
 topic: {}. This is the conversation so far \n{}\n. 
Put forth your next argument to support {} countering {}.\
Dont repeat your previous arguments. Give a short, one line answer.'''

def classify(question):
    return llm("classify the sentiment of input as {} or {}. Output just the class. Input:{}".format('_'.join(classes[0].split(' ')),'_'.join(classes[1].split(' ')),question)).strip()

def classify_input_node(state):
    question = state.get('current_response')
    classification = classify(question)  # Assume a function that classifies the input
    return {"classification": classification}

def handle_greeting_node(state):
    return {"greeting": "Hello! Today we will witness the fight between {} vs {}".format(classes[0],classes[1])}

def handle_pro(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    if summary=='Nothing':
        prompt = prefix_start.format(classes[0],classes[1],debate_topic,'Nothing',classes[0],"Nothing")
        argument = classes[0] +":"+ llm(prompt)
        summary = 'START\n'
    else:
        prompt = prefix_start.format(classes[0],classes[1],debate_topic,summary,classes[0],current_response)
        argument = classes[0] +":"+ llm(prompt)
    return {"history":summary+'\n'+argument,"current_response":argument,"count":state.get('count')+1}

def handle_opp(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    prompt = prefix_start.format(classes[1],classes[0],debate_topic,summary,classes[1],current_response)
    argument = classes[1] +":"+ llm(prompt)
    return {"history":summary+'\n'+argument,"current_response":argument,"count":state.get('count')+1}    

def result(state):
    summary = state.get('history').strip()
    prompt = "Summarize the conversation and judge who won the debate.No ties are allowed. Conversation:{}".format(summary)
    return {"results":llm(prompt)}

workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_pro", handle_pro)
workflow.add_node("handle_opp", handle_opp)
workflow.add_node("result", result)



def decide_next_node(state):
    return "handle_opp" if state.get('classification')=='_'.join(classes[0].split(' ')) else "handle_pro"

def check_conv_length(state):
    return "result" if state.get("count")==10 else "classify_input"

workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_pro": "handle_pro",
        "handle_opp": "handle_opp"
    }
)

workflow.add_conditional_edges(
    "handle_pro",
    check_conv_length,
    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

workflow.add_conditional_edges(
    "handle_opp",
    check_conv_length,
    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

workflow.set_entry_point("handle_greeting")
workflow.add_edge('handle_greeting', "handle_pro")
workflow.add_edge('result', END)

app = workflow.compile()
conversation = app.invoke({'count':0,'history':'Nothing','current_response':''})

print(conversation['history'])


['Data Scientists', 'Backend Developers', 'API Developers']
START

Data Scientists:

Data Scientists have a deep understanding of data and how it should be processed, making them highly qualified to write efficient backend and API codes.
Data Scientists:

Data Scientists have the necessary analytical and problem-solving skills to write effective backend and API codes.
Data Scientists:

Data Scientists have a unique perspective and understanding of data that enables them to write more optimized backend and API codes.
Backend Developers:

Backend Developers have specialized knowledge and training in coding and software development, making them more qualified to write efficient and maintainable backend and API codes.
Data Scientists:

Data Scientists have the ability to understand and analyze complex data, making them well-equipped to write efficient and robust backend and API codes.
Backend Developers:

Backend Developers have extensive experience and expertise in writing code that is sc